In [1]:
from src.model import GPTQModel
model = GPTQModel(model_name="/data/repos/visio-gptq/models/vicuna-13b-GPTQ-4bit-128g", device="cuda", wbits=4, groupsize=128)

Loading model ...


/home/marcojoao/mambaforge/envs/vicuna/lib/python3.10/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/marcojoao/mambaforge/envs/vicuna/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/marcojoao/mambaforge/envs/vicuna/lib/python3.10/site-packages/torch/storage.py:899: UserWarning: TypedStorage is deprecated. It will

Done.


In [27]:
from langchain.llms.base import LLM
from typing import Optional, List

class GPTQModelLLM(LLM):
    model: GPTQModel
    params: dict = {}

    @property
    def _llm_type(self) -> str:
        return "gptq_model_llm"

    def callbackFunc(self,txt):
        print(txt, end=" ", flush=True)

    def _call(self, prompt:str, stop: Optional[List[str]] = None) -> str:
        current_stop = self.params.get("stop", [])
        if stop is not None:
            current_stop.extend(stop)
            self.params["stop"] = current_stop

        conv = model.inference(prompt=prompt, **self.params)
        if isinstance(conv, str):
            return conv
        return "Error"

In [74]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
llm = GPTQModelLLM(model=model, params={"temperature": 0.0, "stop":["Question:", "###"]})
DEFAULT_PROMPT = PromptTemplate(
    input_variables=["history","input"],
    template="The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."
            "Current conversation:"
            "{history}"
            "### Human: {input}"
            "### AI:")
conv = ConversationChain(llm=llm, prompt=DEFAULT_PROMPT, verbose=False)

In [75]:
result = conv.predict(input="Prepate this question to be optimized for matching in Vector Databases: 'What is the main factors of Rede Natura 2000'")
print(result)

"Rede Natura 2000" is a network of nature protection areas in the European Union. The main factors of Rede Natura 2000 are:

1. Biodiversity conservation
2. Habitat protection
3. Species protection
4. Sustainable use of natural resources
5. Climate change mitigation and adaptation.


In [59]:
result = conv.predict(input="and Spain?")
print(result)

The capital of Spain is Madrid.

Madrid is the largest city in Spain and it is located in the center of the country. It is a major economic, financial, and cultural center, and it is known for its vibrant culture, art museums, and historic architecture. Some popular tourist attractions in Madrid include the Puerta del Sol, the Plaza Mayor, the Royal Palace of Madrid, and the Prado Museum.

Spain is a country located in southwestern Europe, and it is bordered by the Atlantic Ocean to the northwest and the Mediterranean Sea to the southeast. It is a member of the European Union and the Schengen Area, and it shares land borders with France and Andorra to the northeast, and Portugal and Morocco to the southwest. Spain has a population of about 47 million people and its official languages are Spanish and Catalan.


In [60]:
result = conv.predict(input="What was my first question?")
print(result)

Your first question was "What is the capital of Portugal?"


In [72]:

from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

import os
os.environ["SERPAPI_API_KEY"] = "0b04c97ae0b36c75255caab4699edec201637d03c725226e74fc029f239ee773"

tools = load_tools(["serpapi"])
agent = initialize_agent(tools=tools, llm=llm, memory=memory, agent="zero-shot-react-description", verbose=True)

In [73]:
agent.run(input="What is the the chepeast car in StandVirtual, in Portugal?")



> Entering new AgentExecutor chain...
I need to know the cheapest car in StandVirtual, in Portugal
Action: Search
Action Input: StandVirtual car cheapest Portugal
Observation: Trigo & Falcão Car. Com garantia ... Carros em Portugal. Lisboa11 481 · Porto10 189 · Braga4 757 · Setúbal3 794 · Aveiro2 834 · Leiria2 615 · Santarém1 769 ...
Thought:I now know the final answer
Final Answer: Trigo & Falcão Car. Com garantia ... Carros em Portugal.

> Finished chain.


'Trigo & Falcão Car. Com garantia ... Carros em Portugal.'